# Import and Init

# 1.1 Python and Stuffs

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Définir le backend souhaité (par exemple, 'tensorflow' ou 'theano')
backend = 'torch'  # Remplacez 'tensorflow' par le backend de votre choix

# Définir la variable d'environnement KERAS_BACKEND
os.environ['KERAS_BACKEND'] = backend

# Importer Keras après avoir défini la variable d'environnement


import PIL
from PIL import Image

In [51]:
import keras


In [20]:
import h5py

# 1.2 Parameters

In [21]:
# filesystem
Data_dir="/home/onyxia/work/AutoEncoder/Data"
Codes_dir="/home/onyxia/work/AutoEncoder/Codes"
Outputs_dir="/home/onyxia/work/AutoEncoder/Outputs"

# Preprocessing 

# Model

2 Load  Data

In [22]:
def retrieve_data(filename) :
    with h5py.File(Data_dir+"/"+filename, 'r') as f:
    # Créer un dataset
       x_train=  f['x_train'][:] 
       y_train=  f['y_train'][:] 
       x_test=  f['x_test'][:] 
       y_test=  f['y_test'][:] 

    return x_train,y_train,x_test,y_test

In [23]:
x_train,y_train,x_test,y_test=retrieve_data("dataset_Images.h5")

In [24]:
x_train.shape

(8000, 28, 28)

3 BUILD MODEL

3.1 ENCODER Model


In [25]:
shape= input_shape+tuple([1])
shape

(0, 5, 1)

In [76]:
def simple_encoder(input_shape,cl,dl,latent_dim,activation="relu",name="encoder",strides=2,padding="same"):
    n_filter=cl['n_filter']
    kernel_size=cl['kernel_size']
    dl_size=dl['dl_size']
    dl_activation=dl["activation"]
    
    inputs=keras.Input(shape= input_shape+tuple([1]),name="Input_Layers")
    
     # add Convolution Layers
    x=keras.layers.Conv2D(n_filter,kernel_size,strides=strides,padding=padding,name="Conv2D_1")(inputs)
    x=keras.layers.Conv2D(n_filter,kernel_size,strides=strides,padding=padding,name="Conv2D_2")(x)
    x=keras.layers.Conv2D(n_filter,kernel_size,strides=strides,padding=padding,name="Conv2D_3")(x)
     # add Flatten layers
    x=keras.layers.Flatten(name="Flatten")(x)
    
     # add Dense Layers
    x=keras.layers.Dense(dl_size,activation=dl_activation,name="Dense_1")(x)
    x=keras.layers.Dense(int(dl_size/2),activation=dl_activation,name="Dense_2")(x)
    x=keras.layers.Dense(int(dl_size/4),activation=dl_activation,name="Dense_3")(x)



    ouputs_Encoder=keras.layers.Dense(latent_dim,activation=activation,name="Dense_output")(x)

    # define model

    encoder=keras.models.Model(inputs,ouputs_Encoder,name="Encoder")

    return encoder

In [77]:
def conv_layer_params(n_filter=32,kernel_size=(3,3)):
    cl={}
    cl["n_filter"]=n_filter
    cl["kernel_size"]=kernel_size
    return cl


In [78]:
def dense_layer_params(dl_size=100,activation="relu"):
    dl={}
    dl["dl_size"]=dl_size
    dl["activation"]=activation
    return dl

In [79]:
cl=conv_layer_params()
dl=dense_layer_params()
input_shape=(28,28)
latent_dim=10
encoder=simple_encoder(input_shape,cl,dl,latent_dim,activation="relu",name="encoder",strides=2,padding="same")

In [80]:
encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ Input_Layers (InputLayer)       │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Conv2D_1 (Conv2D)               │ (None, 14, 14, 32)        │        320 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Conv2D_2 (Conv2D)               │ (None, 7, 7, 32)          │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Conv2D_3 (Conv2D)               │ (None, 4, 4, 32)          │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Flatten (Flatten)               │ (None, 512)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense_1 (Dense)                 │ (None, 100)               │     51,300 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense_2 (Dense)                 │ (None, 50)                │      5,050 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense_3 (Dense)                 │ (None, 25)                │      1,275 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense_output (Dense)            │ (None, 10)                │        260 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 76,701 (299.61 KB)

 Trainable params: 76,701 (299.61 KB)

 Non-trainable params: 0 (0.00 B)